In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 6.4 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
"""
  Import CIFAR-10 data and apply some transformations
"""

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# split training into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

170498071/170498071 [==============================] - 6s 0us/step


In [ ]:
print(X_train.shape, y_train.shape)

(40000, 32, 32, 3) (40000, 1)


In [ ]:
"""
  Basic Deep Neural Network Model
"""

def BasicDNN():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(32,32,3)))
  model.add(keras.layers.Dense(units=1536, activation='relu'))
  model.add(keras.layers.Dense(units=768, activation='relu'))
  model.add(keras.layers.Dense(units=384, activation='relu'))
  model.add(keras.layers.Dense(units=128, activation='relu'))
  model.add(keras.layers.Dense(units=10, activation='softmax'))

  opt = keras.optimizers.Adam(learning_rate=0.01)
  model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

  return model

In [ ]:
basic_dnn = BasicDNN()

In [ ]:
history = basic_dnn.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=256)

Epoch 1/20
157/157 [==============================] - 8s 11ms/step - loss: 4.7021 - accuracy: 0.2241 - val_loss: 1.8948 - val_accuracy: 0.2892
Epoch 2/20
157/157 [==============================] - 1s 7ms/step - loss: 1.8530 - accuracy: 0.3211 - val_loss: 1.7702 - val_accuracy: 0.3564
Epoch 3/20
157/157 [==============================] - 1s 9ms/step - loss: 1.7723 - accuracy: 0.3585 - val_loss: 1.8010 - val_accuracy: 0.3436
Epoch 4/20
157/157 [==============================] - 1s 9ms/step - loss: 1.7332 - accuracy: 0.3719 - val_loss: 1.7255 - val_accuracy: 0.3765
Epoch 5/20
157/157 [==============================] - 2s 12ms/step - loss: 1.7065 - accuracy: 0.3817 - val_loss: 1.7309 - val_accuracy: 0.3818
Epoch 6/20
157/157 [==============================] - 2s 13ms/step - loss: 1.6861 - accuracy: 0.3927 - val_loss: 1.6890 - val_accuracy: 0.3905
Epoch 7/20
157/157 [==============================] - 2s 11ms/step - loss: 1.6792 - accuracy: 0.3931 - val_loss: 1.7148 - val_accuracy: 0.3842
Ep

In [ ]:
basic_dnn.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.5425 - accuracy: 0.4535


[1.5425031185150146, 0.45350000262260437]

In [ ]:
class MyHyperModel(kt.HyperModel):
  def build(self, hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(32,32,3)))
    model.add(keras.layers.Dense(units=hp.Int('layer_1', 1000, 2280, step=128), activation='relu'))
    model.add(keras.layers.Dense(units=768, activation='relu'))
    model.add(keras.layers.Dropout(hp.Float('dropout_1', min_value=0, max_value=1, step=0.2)))
    model.add(keras.layers.Dense(units=384, activation='relu'))
    model.add(keras.layers.Dense(units=128, activation='relu'))
    model.add(keras.layers.Dropout(hp.Float('dropout_2', min_value=0, max_value=1, step=0.2)))
    model.add(keras.layers.Dense(units=10, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    return model
  
  def fit(self, hp, model, *args, **kwargs):
    return model.fit(
        *args,
        batch_size=hp.Int("batch_size", 32, 256, step=32),
        **kwargs,
    )

In [ ]:
dnn_tuner = kt.Hyperband(MyHyperModel(), objective='val_accuracy', overwrite=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
dnn_tuner.search(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[stop_early])

Trial 254 Complete [00h 00m 45s]
val_accuracy: 0.5147500038146973

Best val_accuracy So Far: 0.5350000262260437
Total elapsed time: 01h 14m 10s


In [ ]:
dnn_tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0147 summary
Hyperparameters:
layer_1: 1768
dropout_1: 0.2
dropout_2: 0.0
learning_rate: 0.0001
batch_size: 96
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 4
tuner/round: 4
tuner/trial_id: 0144
Score: 0.5356249809265137

Trial 0242 summary
Hyperparameters:
layer_1: 1768
dropout_1: 0.0
dropout_2: 0.4
learning_rate: 0.0001
batch_size: 128
tuner/epochs: 34
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.5329999923706055

Trial 0234 summary
Hyperparameters:
layer_1: 1384
dropout_1: 0.2
dropout_2: 0.6000000000000001
learning_rate: 0.0001
batch_size: 192
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0230
Score: 0.5301250219345093

Trial 0146 summary
Hyperparameters:
layer_1: 1640
dropout_1: 0.0
dropout_2: 0.2
learning_rate: 0.0001
batch_size: 160
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/brack

In [ ]:
best_model = dnn_tuner.get_best_models()[0]

In [ ]:
best_model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.3618 - accuracy: 0.5423


[1.3617863655090332, 0.5422999858856201]